In [1]:
import pandas as pd

# Load (adjust path if your notebook sits elsewhere)
df = pd.read_csv('C:/Users/Administrator/OneDrive/Documents/YouTube_Trending_Songs_Dashboard/data/youtube-top-100-songs-2025.csv')

# Make column names safe: lowercase, no leading/trailing spaces, spaces -> underscores
df.columns = df.columns.str.strip().str.replace(' ', '_').str.lower()

print("Columns:", df.columns.tolist())
print("Shape (rows, cols):", df.shape)
df.head(3)


Columns: ['title', 'fulltitle', 'description', 'view_count', 'categories', 'tags', 'duration', 'duration_string', 'live_status', 'thumbnail', 'channel', 'channel_url', 'channel_follower_count']
Shape (rows, cols): (100, 13)


,title,fulltitle,description,view_count,categories,tags,duration,duration_string,live_status,thumbnail,channel,channel_url,channel_follower_count
0,ROSÉ & Bruno Mars - APT. (Official Music Video),ROSÉ & Bruno Mars - APT. (Official Music Video),ROSÉ & Bruno Mars - APT.\nDownload/stream: ht...,2009014557,Music,YG Entertainment;YG;와이지;K-pop;BLACKPINK;블랙핑크;블...,173,2:53,False,https://i.ytimg.com/vi_webp/ekr2nIex040/maxres...,ROSÉ,https://www.youtube.com/channel/UCBo1hnzxV9rz3...,19200000
1,"Lady Gaga, Bruno Mars - Die With A Smile (Offi...","Lady Gaga, Bruno Mars - Die With A Smile (Offi...",MAYHEM OUT NOW\nhttp://ladygaga.com \n \nListe...,1324833300,Music,Lady Gaga;Bruno Mars;Interscope;Pop,252,4:12,False,https://i.ytimg.com/vi/kPa7bsKwL-c/maxresdefau...,Lady Gaga,https://www.youtube.com/channel/UC07Kxew-cMIay...,29600000
2,Reneé Rapp - Leave Me Alone (Official Music Vi...,Reneé Rapp - Leave Me Alone (Official Music Vi...,"Listen to “BITE ME”, the new album from Reneé ...",2536628,Music,Reneé Rapp;Interscope Records;Pop,160,2:40,False,https://i.ytimg.com/vi/tiPWzFLiz4A/maxresdefau...,Reneé Rapp,https://www.youtube.com/channel/UCZy4ki_L4bzw9...,408000


# C:/Users/Administrator/OneDrive/Documents/YouTube_Trending_Songs_Dashboard/data/youtube-top-100-songs-2025.csv

In [2]:
# Missing percentage per column
missing_pct = (df.isnull().mean() * 100).round(2).sort_values(ascending=False)
print("Missing % by column:\n", missing_pct)

# Data types
df.dtypes


Missing % by column:
 tags                      15.0
fulltitle                  0.0
description                0.0
view_count                 0.0
title                      0.0
categories                 0.0
duration                   0.0
duration_string            0.0
live_status                0.0
thumbnail                  0.0
channel                    0.0
channel_url                0.0
channel_follower_count     0.0
dtype: float64


title                     object
fulltitle                 object
description               object
view_count                 int64
categories                object
tags                      object
duration                   int64
duration_string           object
live_status                 bool
thumbnail                 object
channel                   object
channel_url               object
channel_follower_count     int64
dtype: object

In [3]:
print("Unique titles:", df['title'].nunique())
print("Unique channels:", df['channel'].nunique())

# Heuristic duplicate check using a subset of columns
dupe_cols = ['title','channel','view_count','duration']
potential_dupes = df.duplicated(subset=[c for c in dupe_cols if c in df.columns]).sum()
print("Potential duplicate rows (by title/channel/view_count/duration):", potential_dupes)


Unique titles: 100
Unique channels: 65
Potential duplicate rows (by title/channel/view_count/duration): 0


In [4]:
num_cols = [c for c in ['view_count','duration','channel_follower_count'] if c in df.columns]
df[num_cols].describe().T


,count,mean,std,min,25%,50%,75%,max
view_count,100.0,1.059103e+08,2.491132e+08,1161.0,18527021.0,37982240.0,1.111073e+08,2.009015e+09
duration,100.0,2.039000e+02,4.426059e+01,120.0,173.0,193.5,2.317500e+02,3.540000e+02
channel_follower_count,100.0,1.612067e+07,1.980038e+07,1.0,613250.0,6320000.0,2.760000e+07,7.620000e+07


In [5]:
print("Top 10 categories:\n", df['categories'].value_counts().head(10))
print("\nTop 10 channels by video count:\n", df['channel'].value_counts().head(10))


Top 10 categories:
 categories
Music             94
People & Blogs     6
Name: count, dtype: int64

Top 10 channels by video count:
 channel
Sabrina Carpenter    7
Tate McRae           4
ROSÉ                 3
Billie Eilish        3
Lady Gaga            3
JENNIE               3
The Weeknd           3
David Guetta         3
Kendrick Lamar       3
LLOUD Official       2
Name: count, dtype: int64


In [6]:
# Average views (overall)
avg_views = df['view_count'].mean()
print("Average views per video:", round(avg_views, 2))

# Most viewed single video
idx = df['view_count'].idxmax()
df.loc[[idx], ['title','channel','view_count','categories','duration','channel_follower_count']]

# Views concentration: what % of total views is in top 10 channels?
channel_views = df.groupby('channel', as_index=False)['view_count'].sum().sort_values('view_count', ascending=False)
total_views = channel_views['view_count'].sum()
top10_views_share = channel_views.head(10)['view_count'].sum() / total_views * 100
print(f"Share of total views captured by Top 10 channels: {top10_views_share:.2f}%")


Average views per video: 105910319.07
Share of total views captured by Top 10 channels: 72.20%


In [7]:
import numpy as np

# If duration looks non-numeric, just inspect values here; we’ll coerce in Step 3.
non_numeric = df['duration'].dtype not in [int, float, 'int64', 'float64']
print("Is duration numeric?", not non_numeric)

# Temporary safe conversion (no overwrite) just for exploration
duration_temp = pd.to_numeric(df['duration'], errors='coerce')
duration_min_temp = duration_temp / 60

# Simple bucket preview
bins = [-0.001, 2, 4, 6, 10, np.inf]
labels = ['<2 min','2–4 min','4–6 min','6–10 min','>10 min']
bucket_preview = pd.cut(duration_min_temp, bins=bins, labels=labels)
print(bucket_preview.value_counts(dropna=False))


Is duration numeric? True
duration
2–4 min     81
4–6 min     18
<2 min       1
6–10 min     0
>10 min      0
Name: count, dtype: int64


In [8]:
# How many rows have tags?
has_tags = df['tags'].notna() & (df['tags'].str.strip() != '')
print("Rows with any tags:", has_tags.sum(), "out of", len(df))

# Average number of tags (rough estimate: split by '|' if that’s the delimiter)
avg_tag_count_est = (
    df.loc[has_tags, 'tags']
      .apply(lambda s: len([t for t in s.split('|') if t.strip()!='']))
      .mean()
)
print("Estimated average tags per video:", round(avg_tag_count_est, 2))


Rows with any tags: 85 out of 100
Estimated average tags per video: 1.0


In [9]:
profile = {
    "rows": [len(df)],
    "unique_titles": [df['title'].nunique()],
    "unique_channels": [df['channel'].nunique()],
    "avg_views": [round(df['view_count'].mean(), 2)],
    "max_views": [int(df['view_count'].max())],
    "missing_view_count_%": [round(df['view_count'].isna().mean()*100, 2)],
    "missing_duration_%": [round(df['duration'].isna().mean()*100, 2)],
    "missing_channel_followers_%": [round(df['channel_follower_count'].isna().mean()*100, 2)],
}
pd.DataFrame(profile).to_csv('../data/_explore_profile_summary.csv', index=False)
print("Saved ../data/_explore_profile_summary.csv")


Saved ../data/_explore_profile_summary.csv
